In [1]:
%load_ext autoreload
%autoreload 2

## Check GPU 

In [2]:
from tensorflow.python.client import device_lib


[x for x in device_lib.list_local_devices() if x.device_type == 'GPU']

[]

## Clone github repository

In [ ]:
! git clone https://github.com/nabacg/aml-nlp-notes.git

In [ ]:
import os 
print(os.getcwd())
[n for n in dir(os) if 'ch' in n]
os.chdir('/content/aml-nlp-notes/language-model')
print(os.getcwd())

## Download data files

## cornell dataset

In [ ]:
! sh download_cornell.sh

## twitter dataset 

In [ ]:
## run in main dir
! git clone https://github.com/Marsan-Ma/chat_corpus
! gzip -d chat_corpus/twitter_en.txt.gz
! mkdir -p /content/aml-nlp-notes/language-model/data/chat_corpus
! mv chat_corpus/twitter_en.txt /content/aml-nlp-notes/language-model/data/chat_corpus

## Open subtitles 

In [ ]:
! sh download_opensubs.sh

## Imports required

In [66]:
import tensorflow as tf
import os
import numpy as np
from seq2seqdataprocessing import load_dataset, word_to_idx
from sklearn.model_selection import train_test_split
from language_model import Encoder, Decoder, train_model, answer 


1.13.1


## Prepare train, test dataset

In [43]:
dataset_name = 'twitter'
input_tensor, target_tensor, dict_index, max_length_inp, max_length_targ = load_dataset(dataset_name = dataset_name, max_sentence_length=20)

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

100%|██████████| 83097/83097 [00:07<00:00, 11151.71it/s]


(22312, 22312, 2480, 2480)

In [44]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
word2idx = dict_index[0]
embedding_dim = 256
units = 1024
vocab_size = len(word2idx)


dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [56]:
! ls ../week5/data

dialogues.tsv    tagged_posts.tsv


## Create encoder, decoder instance

### UPLOAD data/starspace_embeddings.tsv first 

TODO script this! 

In [72]:
embedding, embedding_dim, emb_matrix  = load_embeddings('data/starspace_embeddings.tsv', word2idx)
pretrained_emb = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights = emb_matrix)

Filling pre-trained embeddings...


In [73]:
encoder = Encoder(vocab_size, embedding_dim, units, BATCH_SIZE, emb_matrix)
decoder = Decoder(vocab_size, embedding_dim, units, BATCH_SIZE, emb_matrix)
optimizer = tf.train.AdamOptimizer()

TypeError: super(type, obj): obj must be an instance or subtype of type

## Mount Google drive to save model checkpoints

In [ ]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

## Checkpoints 

In [40]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

### Set checkpoint for Google drive 

In [ ]:
checkpoint_dir = "/content/drive/My Drive/twitter-ds-training"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

### Restore previous checkpoints 

In [29]:
checkpoint_dir = './twitter-ds-training'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [49]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt-12'

In [47]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

ValueError: Shapes (15002, 256) and (2532, 256) are incompatible

## Training 

In [39]:
train_model(encoder, 
            decoder, 
            optimizer,
            dataset, 
            BATCH_SIZE, 
            N_BATCH,
            start_word_index=word_to_idx(word2idx, '<start>'),
            epochs=3,
            save_checkpoint=lambda: checkpoint.save(file_prefix = checkpoint_prefix))

Epoch 1 Batch 0 Loss 4.2950


KeyboardInterrupt: 

## Evaluation

In [23]:
max_length_targ

39

In [25]:
len("quack <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> ".strip().split(' '))

39

In [38]:
answer("what?", encoder, decoder, dict_index, units, max_length_targ)

KeyError: 13758